#### Duplicate Variable Detection and Safe Unification Check


This notebook identifies duplicated or similarly named variables across survey months and determines which ones can be safely unified. It runs after `05_RecordIntegrityCheck.ipynb` and prepares the data **for consistent variable naming before imputation.**


---


**1. Detect Duplicate or Similar Variable Names**
- Groups variables with similar names using fuzzy matching to identify potential duplicates or renamed versions of the same concept.


**2. Compare Category Labels and Meanings**


Checks whether grouped variables truly measure the same thing by comparing:
- category labels
- coding structure
- semantic meaning across survey months


**3. Validate Special Cases**
- Handles exceptions where labels differ but meaning is the same.


**4. Apply Safe Renaming**


Only variables that pass all semantic and coding checks are unified. After validating which variables are safe to unify:
- Standardize variable names
- Standardizes value formats (e.g., lowercase)
- Applies special value harmonization (e.g., standardizing Yes/No).


Outputs are saved in the folder `NEW Renamed Fully Decoded Surveys V2`.


---
**Purpose**


Unifying only truly identical variables:
1. Prevents mixing different concepts under one variable name
2. Preserves data validity for imputation and modeling
3. Ensures consistent variable structures across survey months
4. Reduces bias and prevents distorted analysis results

In [1]:
import json
from pathlib import Path
import os
import pandas as pd
from collections import defaultdict, Counter
from rapidfuzz import fuzz
import re
import csv


# Load config
with open(Path("./data/interim/config.json")) as f:
    cfg = json.load(f)


BASE_PATH = Path(cfg["BASE_PATH"])
INTERIM_DIR = Path(cfg["INTERIM_DIR"])
PROCESSED_DIR = Path(cfg["PROCESSED_DIR"])
LOG_DIR = Path(cfg["LOG_DIR"])
MONTH_ORDER = cfg["MONTH_ORDER"]


# Load inventory
with open(Path(INTERIM_DIR) / "inventory.json") as f:
    inventory = json.load(f)

01 - COLLECT ALL VARIABLES AND MONTH APPEARANCES

This step builds a complete inventory of all column names across all survey files and tracks their presence by month. This is foundational for detecting inconsistencies and harmonizing variable names.

In [2]:
decoded_path = BASE_PATH / "NEW Fully Decoded Surveys"
metadata_path = BASE_PATH / "NEW Metadata Sheet 2 CSV's"


all_columns = []
variable_months = defaultdict(set)


for year in os.listdir(decoded_path):
    year_folder = decoded_path / year
    if not year_folder.is_dir():
        continue


    for file in os.listdir(year_folder):
        if not file.endswith(".CSV"):
            continue


        month = file.split("_")[0].capitalize()
        month_year = f"{month} {year}"
        file_path = year_folder / file


        try:
            df = pd.read_csv(file_path, low_memory=False)
            for col in df.columns:
                col_clean = col.strip()
                all_columns.append(col_clean)
                variable_months[col_clean].add(month_year)
        except Exception as e:
            print(f"[ERROR] {file} -> {e}")


all_columns = sorted(set(all_columns))

02 - CLUSTER SIMILAR VARIABLES (EXCLUSIVE GROUPING)

This block uses fuzzy string matching to group semantically similar variables that may differ due to inconsistent naming. These groups are later used to suggest renaming candidates.

In [3]:
similarity_threshold = 85
filtered_groups = {}
processed_vars = set()


for i, var1 in enumerate(all_columns):
    if var1 in processed_vars:
        continue


    current_group = []


    for j in range(i + 1, len(all_columns)):
        var2 = all_columns[j]
        if var2 in processed_vars:
            continue


        similarity = fuzz.token_sort_ratio(var1.lower(), var2.lower())
        if similarity >= similarity_threshold:
            current_group.append(var2)
            processed_vars.add(var2)


    if current_group:
        filtered_groups[var1] = current_group
        processed_vars.add(var1)

03 - HELPERS

These helper functions support semantic validation by retrieving label values from metadata and normalizing them for accurate comparison. This is essential for verifying whether grouped variables are truly equivalent.

In [4]:
def load_per_month_labels(variable):
    results = defaultdict(set)
    for year in os.listdir(metadata_path):
        year_folder = metadata_path / year
        if not year_folder.is_dir():
            continue
        for file in os.listdir(year_folder):
            if not file.startswith("Sheet2_") or not file.endswith(".csv"):
                continue
            month = file.split("_")[1].capitalize()
            month_year = f"{month} {year}"
            file_path = year_folder / file
            try:
                df = pd.read_csv(file_path, dtype=str).fillna("")
            except Exception:
                continue
            if "Description" not in df.columns or "Label" not in df.columns:
                continue
            match = df[df["Description"].astype(str).str.strip().str.lower() == variable.lower()]
            if not match.empty:
                labels_raw = [str(x).strip() for x in match["Label"].tolist() if str(x).strip() != ""]
                results[month_year].update(labels_raw)
    return results


def normalize(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

04 - PRINT GROUPED ANALYSIS

This section provides a detailed audit of grouped variables, showing their label values across months and highlighting any inconsistencies. It helps validate whether fuzzy-matched variables should be unified and flags potential semantic mismatches.

In [5]:
print("\n============================================================")
print("      GROUPS WITH DETAILED LABEL + MONTH DIFFERENCE CHECK      ")
print("============================================================\n")


group_number = 1


for key, group in filtered_groups.items():
    full_group = sorted(set([key] + group))


    print(f"\n----- Group {group_number} -----")
    print("Variables:")
    for var in full_group:
        month_count = len(variable_months[var])
        print(f"- {var} ({month_count} months)")


    group_labels = {}
    overall_label_union = {}
    pretty_print_map = {}


    for var in full_group:
        per_month = load_per_month_labels(var)
        group_labels[var] = per_month


        overall = set()
        for labels in per_month.values():
            overall.update(labels)
            for label in labels:
                pretty_print_map[normalize(label)] = label


        overall_label_union[var] = overall


        print(f"\nVariable: {var}")
        if not per_month:
            print("Labels: (No labels found in metadata)")


        months_to_show = sorted(
            [m for m in per_month.keys() if m in variable_months[var]],
            key=lambda x: (int(x.split()[-1]), MONTH_ORDER.get(x.split()[0], 99))
        )


        if not months_to_show:
            for m in sorted(variable_months[var], key=lambda x: (int(x.split()[-1]), MONTH_ORDER.get(x.split()[0], 99))):
                print(f"  {m}: (empty)")
        else:
            for month in months_to_show:
                labels = per_month.get(month, set())
                sorted_labels = sorted(list(labels))
                if len(sorted_labels) > 10:
                    label_str = ", ".join(sorted_labels[:10]) + f", ... (+{len(sorted_labels)-10} more)"
                else:
                    label_str = ", ".join(sorted_labels)
                label_str = label_str if labels else "(empty)"
                print(f"  {month}: {label_str}")


    # Fingerprint comparison
    fingerprints = {
        var: frozenset([normalize(s) for s in overall_label_union.get(var, set())])
        for var in full_group
    }


    reference_var = full_group[0]
    reference_set = fingerprints[reference_var]
    same_global_vocab = all(fingerprints[v] == reference_set for v in full_group)


    temporal_mismatches = []
    is_temporally_consistent = True


    for var in full_group:
        var_global_set = fingerprints[var]
        for month in variable_months[var]:
            month_labels_raw = group_labels[var].get(month, set())
            month_labels_normalized = set([normalize(s) for s in month_labels_raw])
            missing_in_month = var_global_set - month_labels_normalized
            if missing_in_month:
                is_temporally_consistent = False
                missing_readable = sorted([pretty_print_map.get(x, x) for x in missing_in_month])
                if len(missing_readable) > 5:
                    missing_str = ", ".join(missing_readable[:5]) + "..."
                else:
                    missing_str = ", ".join(missing_readable)
                temporal_mismatches.append(f"- {var} in {month} is missing: {missing_str}")


    identical = same_global_vocab and is_temporally_consistent
    print("\nIdentical coding scheme?: ", "YES" if identical else "NO")


    if not identical:
        print("Differences found:")
        if temporal_mismatches:
            print(">> TEMPORAL INCONSISTENCIES (Labels missing in specific months):")
            for mismatch in temporal_mismatches[:10]:
                print(mismatch)
            if len(temporal_mismatches) > 10:
                print(f"... and {len(temporal_mismatches) - 10} more months.")


        for var in full_group:
            if var == reference_var:
                continue
            cur_set = fingerprints[var]
            extra_overall = cur_set - reference_set
            missing_overall = reference_set - cur_set


            if extra_overall:
                readable_extra = [pretty_print_map.get(x, x) for x in extra_overall]
                print(f"- {var} has EXTRA overall labels: {', '.join(sorted(readable_extra))}")


            if missing_overall:
                readable_missing = [pretty_print_map.get(x, x) for x in missing_overall]
                print(f"- {var} is MISSING overall labels: {', '.join(sorted(readable_missing))}")


    group_number += 1


      GROUPS WITH DETAILED LABEL + MONTH DIFFERENCE CHECK      


----- Group 1 -----
Variables:
- 2010Urban-RuralFIES (8 months)
- 2015Urban-RuralFIES (13 months)

Variable: 2010Urban-RuralFIES
  January 2018: Rural, Urban
  April 2018: Rural, Urban
  July 2018: Rural, Urban
  October 2018: Rural, Urban
  January 2019: Rural, Urban
  April 2019: Rural, Urban
  July 2019: Rural, Urban
  October 2019: Rural, Urban

Variable: 2015Urban-RuralFIES
  July 2022: Rural, Urban
  August 2022: Rural, Urban
  September 2022: Rural, Urban
  October 2022: Rural, Urban
  November 2022: Rural, Urban
  December 2022: Rural, Urban
  January 2023: Rural, Urban
  February 2023: Rural, Urban
  March 2023: Rural, Urban
  April 2023: Rural, Urban
  May 2023: Rural, Urban
  June 2023: Rural, Urban
  July 2023: Rural, Urban

Identical coding scheme?:  YES

----- Group 2 -----
Variables:
- C08-Overseas Filipino Indicator (17 months)
- C10-Overseas Filipino Indicator (17 months)

Variable: C08-Overseas Filipin

---
After generating the list of variables with similar names and reviewing their detailed value labels across different survey months, a manual inspection should be done to determine which variables truly have identical coding schemes (same category values). Only variables that meet all of the following criteria will be unified under a single standardized variable name:


1. **Same semantic meaning**
2. **Same response structure** (e.g., both categorical, both numeric)
3. **No contradictory values or differences in interpretation**


Variables that share a similar name but differ in category meanings, coding rules, or response formats were not unified. They remain separate to avoid accidental merging of non-equivalent data.


---


**Why Should We Unify Identical Variables?**


1. **Consistency of Meaning:** Only variables measuring the same concept should be merged. If we combine variables with different meanings, the results become inaccurate.


2. **Clean and Valid Categories:** Merging variables with different category labels may introduce contradictory category values. Keeping only identical categories avoids errors in imputation, encoding, and modeling.


3. **Less Bias During Imputation:** Imputation depends on the true distribution of categories. Incorrectly merging variables changes the distribution and produces biased or unrealistic imputed values.


4. **Factor Analysis Reliability:** Factor analysis assumes that each variable represents a consistent measurable concept across all survey months. Incorrectly unified variables may destabilize correlations, weaken KMO/Bartlett, and causes unreliable factor loadings.


---

**SPECIAL CASE 1:** `Available for Work` Variable


During manual inspection, two variants appeared:


- "C27-Available for Work" → values: yes/no
- "C36-Available for Work" → values: yes available/not available


Although the wording of the categories is different, the semantic meaning is identical. Because both survey versions refer to the same concept (availability for work during the reference period), the variable is safe to unify. Values of this variable can be standardized into `"Yes"` and `"No"`.

In [6]:
# ============================================================
# SPECIAL CASE CHECKER FOR SPECIFIC VARIABLES
# ============================================================


VARIABLES_TO_COMPARE = [
    "C27-Available for Work",
    "C36-Available for Work"
]


print("\n============================================================")
print("      SPECIAL CASE: Available for Work Variables Comparison")
print("============================================================\n")


group_labels = {}
overall_label_union = {}
pretty_print_map = {}


# 1. Load per-month label sets using your existing helper function
for var in VARIABLES_TO_COMPARE:
    per_month = load_per_month_labels(var)
    group_labels[var] = per_month


    union_set = set()
    for labels in per_month.values():
        union_set.update(labels)


        # Populate pretty print map to preserve original text
        for label in labels:
            pretty_print_map[normalize(label)] = label


    overall_label_union[var] = union_set


# 2. Print presence summary based on variable_months
print("Variables:")
for var in VARIABLES_TO_COMPARE:
    month_count = len(variable_months[var])
    print(f"- {var} ({month_count} months)")


# 3. Show per-month labels
print("\n===== PER-MONTH LABEL VALUES =====")


for var in VARIABLES_TO_COMPARE:
    print(f"\nVariable: {var}")


    per_month = group_labels.get(var, {})
    months_to_show = sorted(
        variable_months[var],
        key=lambda x: (int(x.split()[1]), MONTH_ORDER.get(x.split()[0], 99))
    )


    if not months_to_show:
        print("  (No months found in metadata)")
        continue


    for month in months_to_show:
        labels = per_month.get(month, set())
        if labels:
            sorted_labels = sorted(list(labels))
            label_str = ", ".join(sorted_labels)
        else:
            label_str = "(empty)"
        print(f"  {month}: {label_str}")


# 4. Create normalized fingerprints for comparison
fingerprints = {
    var: frozenset([normalize(s) for s in overall_label_union[var]])
    for var in VARIABLES_TO_COMPARE
}


ref_var = VARIABLES_TO_COMPARE[0]
other_var = VARIABLES_TO_COMPARE[1]


ref_set = fingerprints[ref_var]
other_set = fingerprints[other_var]


# 5. Check coding scheme equivalence
identical = (ref_set == other_set)


print("\n===== CODING SCHEME COMPARISON =====")
print("Identical coding scheme?:", "YES" if identical else "NO")


# 6. Report differences if not identical
if not identical:
    print("Differences found:")


    extra = other_set - ref_set
    missing = ref_set - other_set


    if extra:
        readable = [pretty_print_map.get(x, x) for x in extra]
        print(f"- {other_var} has EXTRA labels: {', '.join(sorted(readable))}")


    if missing:
        readable = [pretty_print_map.get(x, x) for x in missing]
        print(f"- {other_var} is MISSING labels: {', '.join(sorted(readable))}")


      SPECIAL CASE: Available for Work Variables Comparison

Variables:
- C27-Available for Work (17 months)
- C36-Available for Work (17 months)

===== PER-MONTH LABEL VALUES =====

Variable: C27-Available for Work
  August 2022: no, yes
  September 2022: no, yes
  November 2022: no, yes
  December 2022: no, yes
  February 2023: no, yes
  March 2023: no, yes
  May 2023: no, yes
  June 2023: no, yes
  August 2023: no, yes
  September 2023: no, yes
  November 2023: no, yes
  December 2023: no, yes
  February 2024: no, yes
  March 2024: no, yes
  May 2024: no, yes
  June 2024: no, yes
  August 2024: no, yes

Variable: C36-Available for Work
  January 2018: not available, yes available
  April 2018: not available, yes available
  July 2018: not available, yes available
  October 2018: not available, yes available
  January 2019: not available, yes available
  April 2019: not available, yes available
  July 2019: not available, yes available
  October 2019: not available, yes available
  

**SPECIAL CASE 2:** `Work Indicator` Variable


During manual inspection, the “Work Indicator” group contains three variables, but only **C09-Work Indicator** and **C11-Work Indicator** have **identical coding schemes**. The third variable, C09A-Work Indicator, measures a completely different concept, so it will not be included in the unification.


- "C09-Work Indicator" → values: yes/no
- "C11-Work Indicator" → values: yes/no/YES/NO

In [7]:
# ============================================================
# SPECIAL CASE: Work Indicator Variables Comparison
# ============================================================


VARIABLES_TO_COMPARE = [
    "C09-Work Indicator",
    "C11-Work Indicator"
]


print("\n============================================================")
print("      SPECIAL CASE: Work Indicator Variables Comparison")
print("============================================================\n")


group_labels = {}
overall_label_union = {}
pretty_print_map = {}


# 1. Load per-month label sets using your existing helper function
for var in VARIABLES_TO_COMPARE:
    per_month = load_per_month_labels(var)
    group_labels[var] = per_month


    union_set = set()
    for labels in per_month.values():
        union_set.update(labels)


        # Populate pretty print map to preserve original text
        for label in labels:
            pretty_print_map[normalize(label)] = label


    overall_label_union[var] = union_set


# 2. Print presence summary based on variable_months
print("Variables:")
for var in VARIABLES_TO_COMPARE:
    # Uses the variable_months dictionary from your inventory step
    month_count = len(variable_months.get(var, []))
    print(f"- {var} ({month_count} months)")


# 3. Show per-month labels
print("\n===== PER-MONTH LABEL VALUES =====")


for var in VARIABLES_TO_COMPARE:
    print(f"\nVariable: {var}")


    per_month = group_labels.get(var, {})
    # Sort months using your MONTH_ORDER config
    months_to_show = sorted(
        variable_months.get(var, []),
        key=lambda x: (int(x.split()[1]), MONTH_ORDER.get(x.split()[0], 99))
    )


    if not months_to_show:
        print("  (No months found in metadata)")
        continue


    for month in months_to_show:
        labels = per_month.get(month, set())
        if labels:
            sorted_labels = sorted(list(labels))
            label_str = ", ".join(sorted_labels)
        else:
            label_str = "(empty)"
        print(f"  {month}: {label_str}")


# 4. Create normalized fingerprints for comparison
fingerprints = {
    var: frozenset([normalize(s) for s in overall_label_union.get(var, set())])
    for var in VARIABLES_TO_COMPARE
}


ref_var = VARIABLES_TO_COMPARE[0]
other_var = VARIABLES_TO_COMPARE[1]


ref_set = fingerprints[ref_var]
other_set = fingerprints[other_var]


# 5. Check coding scheme equivalence
identical = (ref_set == other_set)


print("\n===== CODING SCHEME COMPARISON =====")
print("Identical coding scheme?:", "YES" if identical else "NO")


# 6. Report differences if not identical
if not identical:
    print("Differences found between Work Indicator versions:")


    extra = other_set - ref_set
    missing = ref_set - other_set


    if extra:
        readable = [pretty_print_map.get(x, x) for x in extra]
        print(f"- {other_var} has EXTRA labels: {', '.join(sorted(readable))}")


    if missing:
        readable = [pretty_print_map.get(x, x) for x in missing]
        print(f"- {other_var} is MISSING labels: {', '.join(sorted(readable))}")


      SPECIAL CASE: Work Indicator Variables Comparison

Variables:
- C09-Work Indicator (17 months)
- C11-Work Indicator (17 months)

===== PER-MONTH LABEL VALUES =====

Variable: C09-Work Indicator
  August 2022: no, yes
  September 2022: no, yes
  November 2022: no, yes
  December 2022: no, yes
  February 2023: no, yes
  March 2023: no, yes
  May 2023: no, yes
  June 2023: no, yes
  August 2023: no, yes
  September 2023: no, yes
  November 2023: no, yes
  December 2023: no, yes
  February 2024: no, yes
  March 2024: no, yes
  May 2024: no, yes
  June 2024: no, yes
  August 2024: no, yes

Variable: C11-Work Indicator
  January 2018: no, yes
  April 2018: no, yes
  July 2018: no, yes
  October 2018: no, yes
  January 2019: no, yes
  April 2019: no, yes
  July 2019: no, yes
  October 2019: no, yes
  July 2022: NO, YES
  October 2022: NO, YES
  January 2023: NO, YES
  April 2023: NO, YES
  July 2023: NO, YES
  October 2023: NO, YES
  January 2024: NO, YES
  April 2024: NO, YES
  July 2

#### Renaming Variables + Automated saving into a new folder

In [8]:
import os
import pandas as pd
from pathlib import Path


# ===============================================================
# UNIVERSAL VALUE STANDARDIZATION FUNCTION
# ===============================================================
def standardize_categorical_values(df):
    """
    Standardize all object-type (categorical) values:
      - lowercase
      - strip whitespace
      - convert 'nan', '', 'none' to actual None
    """
    for col in df.select_dtypes(include='object').columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.lower()
            .replace({"nan": None, "none": None, "": None})
        )
    return df




# ===============================================================
# USER-DEFINED INPUT: THE RENAMING INSTRUCTION MAP
# ===============================================================
RENAMING_MAP = {
    "Work Indicator": [
        "C09-Work Indicator",
        "C11-Work Indicator"
    ],
    "Available for Work": [
        "C27-Available for Work",
        "C36-Available for Work"
    ],
    "Urban-RuralFIES": [
        "2010Urban-RuralFIES",
        "2015Urban-RuralFIES"
    ],
    "Location of Work (Province, Municipality)": [
        "C11 - Location of Work (Province, Municipality)",
        "C11-Location of Work (Province, Municipality)",
        "C12A - Location of Work (Province, Municipality)"
    ],
    "Normal Working Hours per Day": [
        "C17-Normal Working Hours per Day",
        "C18-Normal Working Hours per Day"
    ],
    "Want More Hours of Work": [
        "C19-Want More Hours of Work",
        "C20-Want More Hours of Work"
    ],
    "Look for Additional Work": [
        "C20-Look for Additional Work",
        "C21-Look for Additional Work"
    ],
    "Other Job Indicator": [
        "C22-Other Job Indicator",
        "C26-Other Job Indicator"
    ],
    "Total Hours Worked for all Jobs": [
        "C23-Total Hours Worked for all Jobs",
        "C28-Total Hours Worked for all Jobs"
    ],
    "Looked for Work or Tried to Establish Business During the Past Week": [
        "C25-Looked for Work or Tried to Establish Business during the past week",
        "C30-Looked for Work or Tried to Establish Business during the past week"
    ],
    "First Time to Look for Work": [
        "C25B - First time to look for work",
        "C31-First Time to Look for Work"
    ],
    "Previous Job Indicator": [
        "C28-Previous Job Indicator",
        "C38-Previous Job Indicator"
    ],
    "Previous Occupation": [
        "C31-Previous Occupation",
        "C40-Previous Occupation"
    ],
    "Kind of Business (Past Quarter)": [
        "C33-Kind of Business (past quarter)",
        "C43-Kind of Business (past quarter)"
    ],
    "Province": [
        "Province",
        "province"
    ],
    "Province Recode": [
        "Province Recode",
        "province_recode"
    ]
}


# ===============================================================
# VALUE HARMONIZATION/LOOKUP FOR SPECIFIC VARIABLE
# ===============================================================
AVAILABLE_FOR_WORK_HARMONIZATION = {
    "no": "no",
    "yes": "yes",
    "not available": "no",
    "yes available": "yes"
}


# ===============================================================
# PATHS AND CONFIGURATION
# ===============================================================
SOURCE_FOLDER = "NEW Fully Decoded Surveys"
DESTINATION_FOLDER = "NEW Renamed Fully Decoded Surveys V2"


decoded_path = BASE_PATH / SOURCE_FOLDER
renamed_path = BASE_PATH / DESTINATION_FOLDER
renamed_path.mkdir(parents=True, exist_ok=True)
print(f"Output folder created/verified: {renamed_path}\n")


# --- Reverse renaming map ---
REVERSE_RENAMING_MAP = {
    old.strip(): new.strip()
    for new, old_list in RENAMING_MAP.items()
    for old in old_list
}


# ===============================================================
# RENAME + HARMONIZE FUNCTION (MODIFIED)
# ===============================================================
def rename_and_save_survey(source_filepath, dest_filepath, renaming_map, harmonization_map):
    try:
        df = pd.read_csv(source_filepath, low_memory=False)


        # Step 1: Standardize ALL categorical values BEFORE renaming
        df = standardize_categorical_values(df)


        columns_to_rename = {}


        # Step 2: Special Harmonization for Available-for-Work
        available_cols = [
            col for col in df.columns
            if col.strip() in RENAMING_MAP["Available for Work"]
        ]
        if available_cols:
            lookup = {k.lower(): v for k, v in harmonization_map.items()}
            for col in available_cols:
                df[col] = df[col].map(lookup).fillna(df[col])


        # Step 3: Rename columns based on mapping
        for col in df.columns:
            clean = col.strip()
            if clean in renaming_map:
                columns_to_rename[col] = renaming_map[clean]


        df.rename(columns=columns_to_rename, inplace=True)


        # Step 4: Save cleaned CSV
        df.to_csv(dest_filepath, index=False)


        return len(columns_to_rename)


    except Exception as e:
        print(f"[ERROR] Cannot process {source_filepath}: {e}")
        return -1




# ===============================================================
# BATCH PROCESSING
# ===============================================================
def run_batch_renaming(source_root, dest_root, renaming_map, harmonization_map):
    total_files_processed = 0
    total_columns_unified = 0


    print("=== STARTING BATCH RENAMING ===")
    print(f"Source:      {source_root}")
    print(f"Destination: {dest_root}")
    print("-" * 60)


    for year_folder in sorted(source_root.iterdir()):
        if not year_folder.is_dir():
            continue


        output_year_folder = dest_root / year_folder.name
        output_year_folder.mkdir(exist_ok=True)


        for file in year_folder.iterdir():
            if file.suffix.lower() != ".csv":
                continue


            dest_file = output_year_folder / file.name
            renamed_count = rename_and_save_survey(
                file, dest_file, renaming_map, harmonization_map
            )


            if renamed_count >= 0:
                total_files_processed += 1
                total_columns_unified += renamed_count
                status = (
                    f"Unified {renamed_count} column(s)"
                    if renamed_count > 0
                    else "Saved (No changes)"
                )
                print(f"[OK] {year_folder.name}/{file.name}: {status}")
            else:
                print(f"[FAIL] {year_folder.name}/{file.name}")


    print("-" * 60)
    print("BATCH RENAMING COMPLETE!")
    print(f"Total Files Processed: {total_files_processed}")
    print(f"Total Columns Unified: {total_columns_unified}")
    print(f"Output saved in: {dest_root}")




# ===============================================================
# EXECUTION
# ===============================================================
if __name__ == "__main__":
    run_batch_renaming(
        decoded_path,
        renamed_path,
        REVERSE_RENAMING_MAP,
        AVAILABLE_FOR_WORK_HARMONIZATION
    )

Output folder created/verified: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\NEW Renamed Fully Decoded Surveys V2

=== STARTING BATCH RENAMING ===
Source:      G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\NEW Fully Decoded Surveys
Destination: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\NEW Renamed Fully Decoded Surveys V2
------------------------------------------------------------
[OK] 2018/JANUARY_2018.CSV: Unified 15 column(s)
[OK] 2018/OCTOBER_2018.CSV: Unified 15 column(s)
[OK] 2018/JULY_2018.CSV: Unified 15 column(s)
[OK] 2018/APRIL_2018.CSV: Unified 15 column(s)
[OK] 2019/JULY_2019.CSV: Unified 13 column(s)
[OK] 2019/JANUARY_2019.CSV: Unified 13 column(s)
[OK] 2019/OCTOBER_2019.CSV: Unified 13 column(s)
[OK] 2019/APRIL_2019.CSV: Unified 13 column(s)
[OK] 2022/JANUARY_2022.csv: Unified 13 column(s)
[OK] 2022/OCTOBER_2022.CSV: Unified 13 column(s)
[OK] 2022/APRIL_2022.csv: Un

#### Duplicate Column Checking

In [9]:
import pandas as pd
from pathlib import Path


# --- Path Configuration ---
RENAMED_V2_ROOT = BASE_PATH / "NEW Renamed Fully Decoded Surveys V2"


def run_duplicate_header_check(root_path):
    total_files = 0
    files_with_dupes = []


    print("\n--- STARTING DUPLICATE HEADER CHECK ---")
    print("-" * 50)


    # Walk through the years and files
    for year_folder in sorted(root_path.iterdir()):
        if not year_folder.is_dir():
            continue


        for file in year_folder.glob("*.csv"):
            total_files += 1
           
            # Read only the header row for speed
            headers = pd.read_csv(file, nrows=0).columns.tolist()
           
            # Check for duplicates
            seen = set()
            dupes = set([x for x in headers if x in seen or seen.add(x)])
           
            if dupes:
                files_with_dupes.append((f"{year_folder.name}/{file.name}", dupes))


    # --- Output Formatting ---
    if not files_with_dupes:
        print("-" * 50)
        print("DUPLICATION CHECK COMPLETE.")
        print(f"Total files checked: {total_files}")
        print("No duplicate headers found across all files.")
    else:
        print(f"  ALERT: Found {len(files_with_dupes)} files with duplicate headers.")
        print("-" * 50)
        for filename, dupe_vars in files_with_dupes:
            print(f"File: {filename}")
            print(f"Duplicate Columns: {', '.join(dupe_vars)}")
            print("-" * 20)
        print("\nDUPLICATION CHECK COMPLETE.")
        print(f"Total files checked: {total_files}")
        print(f"Resolution Required: {len(files_with_dupes)} files.")


# --- Execution ---
if RENAMED_V2_ROOT.exists():
    run_duplicate_header_check(RENAMED_V2_ROOT)
else:
    print(f" Error: Path not found: {RENAMED_V2_ROOT}")


--- STARTING DUPLICATE HEADER CHECK ---
--------------------------------------------------
--------------------------------------------------
DUPLICATION CHECK COMPLETE.
Total files checked: 40
No duplicate headers found across all files.


#### Duplicate Column and Value Checking for "Available for Work" variable

In [10]:
import os
import pandas as pd
from pathlib import Path


# ===============================================================
# PATHS AND CONFIGURATION
# ===============================================================
# Ensure it points to your newest unified folder
RENAMED_V2_ROOT = BASE_PATH / "NEW Renamed Fully Decoded Surveys V2"
TARGET_VARIABLE = "Available for Work"


def validate_available_for_work(source_root):
    """
    Checks for the consolidation integrity of the target variable,
    verifies its label set, and checks for any duplicated headers.
    """
    if not source_root.exists():
        print(f"[ERROR] Source folder not found: {source_root}")
        return


    global_label_set = set()
    total_files_checked = 0
    files_with_duplicates = 0
   
    print(f"--- STARTING DUPLICATE HEADER CHECK FOR \"{TARGET_VARIABLE}\" VARIABLE ---")
    print("-" * 50)
    print("-" * 50)


    # 1. Iterate through files and check integrity
    for year_folder in sorted(source_root.iterdir()):
        if not year_folder.is_dir():
            continue
       
        for file in year_folder.glob("*.csv"):
            try:
                # Optimized: Read only the header first to check for duplicates
                # This prevents loading the whole file if a header error exists
                df_headers = pd.read_csv(file, nrows=0)
                total_files_checked += 1
               
                # --- A. Check for any Duplicate Headers in the file ---
                # Check if total columns equals unique columns
                if len(df_headers.columns) != len(set(df_headers.columns)):
                    files_with_duplicates += 1
               
                # --- B. Specific Target Variable Check ---
                if TARGET_VARIABLE in df_headers.columns:
                    # If the target variable itself appears more than once, count it
                    if list(df_headers.columns).count(TARGET_VARIABLE) > 1:
                        # We don't increment twice if the file already failed the duplicate check above
                        pass
                   
                    # Extract unique labels to verify harmonization
                    # We load only the specific column to save memory
                    df_col = pd.read_csv(file, usecols=[TARGET_VARIABLE], low_memory=False)
                    labels = df_col[TARGET_VARIABLE].dropna().unique()
                   
                    # Clean and store labels
                    cleaned_labels = {str(l).strip() for l in labels if str(l).strip() != ''}
                    global_label_set.update(cleaned_labels)
                   
            except Exception as e:
                print(f"[ERROR] Failed to process {file.name}: {e}")


    # 2. Print Final Summary in the requested format
    sorted_labels = sorted(list(global_label_set))
   
    print(f"{TARGET_VARIABLE}")
    print(f"Labels: {', '.join(sorted_labels) if sorted_labels else '(No data found)'}")
   
    print("\n" + "=" * 50)
    print(f"Total files checked: {total_files_checked}")
   
    if files_with_duplicates > 0:
        print(f"Duplicate headers found in {files_with_duplicates} files (Requires consolidation).")
    else:
        print("No duplicate headers found across all files.")


# --- EXECUTION ---
if __name__ == "__main__":
    validate_available_for_work(RENAMED_V2_ROOT)



--- STARTING DUPLICATE HEADER CHECK FOR "Available for Work" VARIABLE ---
--------------------------------------------------
--------------------------------------------------
Available for Work
Labels: no, yes

Total files checked: 40
No duplicate headers found across all files.
